In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 6it [00:00, 31.48it/s, center_shift=0.000072, iteration=6, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Trying 50 clusters for layer blocks.0.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 29it [00:03,  9.02it/s, center_shift=0.000000, iteration=29, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Trying 50 clusters for layer blocks.1.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 21it [00:02,  8.62it/s, center_shift=0.000000, iteration=21, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Trying 50 clusters for layer blocks.2.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 33it [00:03,  8.75it/s, center_shift=0.000000, iteration=33, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Getting cluster scores for lattice
Getting scores for layers 0 to 1
Getting scores for layers 1 to 2
CLUSTER SCORES [array([[  1.33780391,   0.35734555,   1.47031524, ...,   1.30452515,
          1.51189433,   1.51227541],
       [  0.56054979, 100.15075077,   0.60508765, ...,   0.54565826,
          0.55638145,   0.54128113],
       [  1.09367666,   0.29334867,   1.1919706 , ...,   1.06548217,
          1.12496345,   1.10018434],
       ...,
       [  1.26702955,   0.33881838,   1.3982605 , ...,   1.2367633 ,
          1.28468915,   1.24846999],
       [  1.10228281,   0.2947915 ,   1.17996213, ...,   1.06977477,
          1.07650954,   1.04313631],
       [  1.3247038 ,   0.35351438,   1.45270499, ...,   1.29077459,
          1.35730123,   1.31858246]]), array([[1.21859869, 1.23590589, 1.25002714, ..., 1.28484193, 1.16459202,
        1.19201909],
       [1.68219653, 1.70629169, 1.70139011, ..., 1.7121168 , 1.65469421,
        1.69979852],
       [1

In [3]:
import utils
importlib.reload(utils)
utils.top_k_dag_paths_dynamic(decomp.lattice_scores, k=200_000)

[([1, 1, 12, 0], 131.93141977712884),
 ([1, 1, 30, 0], 103.25429882965545),
 ([1, 1, 45, 0], 101.86329505173613),
 ([1, 1, 47, 0], 101.86286757247171),
 ([1, 1, 26, 0], 101.86143310736908),
 ([1, 1, 1, 0], 101.85704246184258),
 ([1, 1, 22, 0], 101.85380412094194),
 ([1, 1, 20, 0], 101.85369298992975),
 ([1, 1, 44, 0], 101.85265471458996),
 ([1, 1, 2, 0], 101.8521408850677),
 ([1, 1, 39, 0], 101.85092444241843),
 ([1, 1, 49, 0], 101.85054929034145),
 ([1, 1, 28, 0], 101.84966006553874),
 ([1, 1, 23, 0], 101.84636605485085),
 ([1, 1, 21, 0], 101.84512652950924),
 ([1, 1, 38, 0], 101.84416214550073),
 ([1, 1, 46, 0], 101.84381996473688),
 ([1, 1, 40, 0], 101.84316071325944),
 ([1, 1, 33, 0], 101.84201564998564),
 ([1, 1, 32, 0], 101.84134611810411),
 ([1, 1, 24, 0], 101.8413243897618),
 ([1, 1, 6, 0], 101.83977686614445),
 ([1, 1, 11, 0], 101.83911865480184),
 ([1, 1, 25, 0], 101.83756471510412),
 ([1, 1, 43, 0], 101.83521737430993),
 ([1, 1, 15, 0], 101.83322109920134),
 ([1, 1, 0, 0], 1

In [4]:

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
score_path = [20, 1, 12]
"""
Paths which seem to have meaning

[1, 3, 25]: relational words like "if" "in" "soon"
[1, 3, 10]: relational word but now it is more specific to "to"
"""

to_score = [d[:100] for d in ds][:100]
scores = decomp.score(
    to_score,
    score_path=score_path
)

DS (3, 2816, 512) 2816
(2816, 3, 512) [26, 16, 27]


In [5]:
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!

# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=[[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord],
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [6]:
scores_per_token_set, scores_per_token_set[top_args]

(array([0.0614259 , 0.06208046, 0.06179711, 0.06463689, 0.06328799,
        0.08383776, 0.06325578, 0.07050524, 0.06918843, 0.0606299 ,
        0.06293083, 0.06211471, 0.06433218, 0.06288906, 0.06247019,
        0.06255169, 0.06200428, 0.06282957, 0.07119591, 0.07102124,
        0.06434829, 0.06491407, 0.09131309, 0.06348113, 0.06273388,
        0.06508535, 0.0613324 , 0.08773295, 0.06420814, 0.06325271,
        0.07627933, 0.06211471, 0.06207982, 0.06254682, 0.06275702,
        0.06310343, 0.06356595, 0.06261998, 0.06208216, 0.06304062,
        0.06243071, 0.06514088, 0.06303531, 0.06158365, 0.06588339,
        0.0630726 , 0.0624725 , 0.06328597, 0.06363885, 0.06176336,
        0.06190457, 0.06202984, 0.06282941, 0.07898954, 0.08469404,
        0.0674923 , 0.06258151, 0.06536476, 0.06269514, 0.06460852,
        0.06264381, 0.0607405 , 0.06335868, 0.07081147, 0.06940678,
        0.06539505, 0.06282235, 0.0623139 , 0.06859034, 0.06444885,
        0.06206938, 0.07585537, 0.06046769, 0.06

In [7]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_282569/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
